In [ ]:
!pip install scrapy

     |████████████████████████████████| 254 kB 5.2 MB/s 
     |████████████████████████████████| 3.2 MB 66.6 MB/s 
     |████████████████████████████████| 65 kB 4.0 MB/s 
     |████████████████████████████████| 55 kB 3.3 MB/s 
     |████████████████████████████████| 3.1 MB 53.5 MB/s 
     |████████████████████████████████| 47 kB 4.9 MB/s 
     |████████████████████████████████| 3.6 MB 66.6 MB/s 
     |████████████████████████████████| 251 kB 71.8 MB/s 
     |████████████████████████████████| 74 kB 3.7 MB/s 
  Created wheel for protego: filename=Protego-0.1.16-py3-none-any.whl size=7782 sha256=7f5015390f2165925c5ae302b5bab1899fa5345954caa1aa925bbd7471fc0f04
  Stored in directory: /root/.cache/pip/wheels/ca/44/01/3592ccfbcfaee4ab297c4097e6e9dbe1c7697e3531a39877ab
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-py3-none-any.whl size=11515 sha256=adbdb1c29d007d2a5c4e0a2d3980ab0815401cb220096bfe8001d6728eec7362
  Stored in directory: /root/.cache/pip/wheels/2d/18/21/3c6a732eaa

In [ ]:
import re
import nltk
import requests
from scrapy import Selector
from datetime import datetime
from nltk.corpus import stopwords
nltk.download('stopwords')

def get_html(url):
  response = requests.get(url)
  content = response.content
  return content

def get_url(sel):
  url = sel.css("div.article__list__title > h3.article__title > a::attr(href)").extract_first() # dapetin link untuk masuk ke inner page
  return url

def parse_content(sel):
  data = {}
  
  # extract scrape_time
  data['scrape_time'] = str(datetime.now())

  # extract story_source
  story_source = sel.css("div.read__time > a::attr(href)").extract_first()
  data['story_source'] = story_source

  # extract story_release_date
  story_release_date = sel.css("div.read__time::text").extract_first()
  story_release_date = story_release_date.replace(" - ", "")
  data['story_release_date'] = story_release_date

  # extract story_title
  story_title = sel.css("h1.read__title::text").extract_first()
  data['story_title'] = story_title

  # extract story_url
  story_url = sel.css("#urlpage::attr(value)").extract_first()
  data['story_url'] = story_url

  # extract content
  contents = sel.css("div.read__content > div.clearfix > p::text").extract()
  merged = ' '.join(contents)
  rm_special_char = re.sub(r'[^\w\s]', '', merged)
  rm_num = re.sub(r'[0-9]+', '', rm_special_char)
  half_clean = rm_num.lower().split()

  en_stops = set(stopwords.words('indonesian'))
  clean = []
  for word in half_clean: 
    if word not in en_stops:
      clean.append(word)

  wordfreq = {}
  for word in clean:
    if word not in wordfreq:
        wordfreq[word] = 0 
    wordfreq[word] += 1

  most_common_words = list(dict(sorted(wordfreq.items(), key=lambda kv:(kv[1], kv[0]), reverse=True)[:5]).keys())

  data["most_common_words"] = most_common_words

  return data#, merged

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
html_page = get_html("https://www.kompas.com/tag/jawa+barat")
html_sel = Selector(text=html_page)

news = html_sel.css("div.article__list") # ambil section dari outer page (misal ada 10 berita)

scraps = []
# mergeds = []
for sel in news:
  url = get_url(sel)
  content_page = get_html(url) # langsung masuk ke masing-masing berita
  content_sel = Selector(text=content_page)
  data = parse_content(content_sel) #data, merged = parse_content(content_sel)

  scraps.append(data)
  # mergeds.append(merged)

In [ ]:
scraps

[{'most_common_words': ['minuman',
   'tradisional',
   'terbuat',
   'menyehatkan',
   'jawa'],
  'scrape_time': '2022-01-05 15:30:10.436827',
  'story_release_date': 'Kompas.com01/01/2022, 15:26 WIB',
  'story_source': None,
  'story_title': 'Siswa, Ini Lho 5 Minuman Tradisional Khas Jabar yang Enak dan Sehat',
  'story_url': 'https://edukasi.kompas.com/read/2022/01/01/152620171/siswa-ini-lho-5-minuman-tradisional-khas-jabar-yang-enak-dan-sehat'},
 {'most_common_words': ['bekasi', 'pengawalan', 'orang', 'nggak', 'kota'],
  'scrape_time': '2022-01-05 15:30:10.637225',
  'story_release_date': '01/01/2022, 10:03 WIB',
  'story_source': 'https://www.kompas.com',
  'story_title': 'Pengakuan Anggota Dishub Bekasi yang Kawal Mobil Mewah dan Lawan Arah: Bukan Pejabat, Warga Biasa',
  'story_url': 'https://regional.kompas.com/read/2022/01/01/100326378/pengakuan-anggota-dishub-bekasi-yang-kawal-mobil-mewah-dan-lawan-arah-bukan'},
 {'most_common_words': ['penduduk', 'indonesia', 'data', 'sensus

In [ ]:
import pprint

pp = pprint.PrettyPrinter(indent=2)

for scrap in scraps:
  pp.pprint(scrap)

In [ ]:
import json

with open("kompas.json", "w") as file:
  file.write(json.dumps(scraps))

for testing purposes, variable merged dalam parse_content() harus dikeluarkan, dan dimasukan kedalam list bernama mergeds bersamaan dengan scraps (diluar fungsi)

In [ ]:
sample = mergeds[0]

In [ ]:
sample

" - Di tengah menjamurnya berbagai kuliner kekinian, Indonesia mempunyai beragam   yang tak kalah enak. Seperti gudeg yang menjadi kuliner khas dari Yogyakarta, rawon dari Jawa Timur dan banyak makanan tradisional lainnya. Tak hanya makanan, banyak minuman tradisional yang tidak hanya menyegarkan tapi juga menyehatkan bagi tubuh. Di Provinsi Jawa Barat, ada beberapa minuman khas yang wajib dicoba. Merangkum dari akun Instagram Dinas Pendidikan Jawa Barat, Sabtu (1/1/2022), dijelaskan beberapa   yang menggugah selera dan menyehatkan. Yuk baca informasi berikut ini. Minuman   ini terbuat dari gula aren dan santan kelapa ini biasanya disajikan hangat dan diminum saat cuaca dingin. Untuk menambah kenikmatan dan khasiat bagi tubuh, bajigur bisa dicampur dengan sedikit jahe dan garam. Bajigur baik bagi tubuh karena dibuat dari bahan-bahan alami yang menyehatkan. Minuman ini memiliki aroma cukup kuat karena memakai rempah-rempah yang menyehatkan. Jahe dan gula merah merupakan bahan dasar band

In [ ]:
rm_special_char = re.sub(r'[^\w\s]', '', sample)
rm_num = re.sub(r'[0-9]+', '', rm_special_char)
half_clean = rm_num.lower().split()
en_stops = set(stopwords.words('indonesian'))
clean = []
for stop in half_clean: 
  if stop not in en_stops:
    clean.append(stop)

In [ ]:
clean

['menjamurnya',
 'kuliner',
 'kekinian',
 'indonesia',
 'beragam',
 'kalah',
 'enak',
 'gudeg',
 'kuliner',
 'khas',
 'yogyakarta',
 'rawon',
 'jawa',
 'timur',
 'makanan',
 'tradisional',
 'makanan',
 'minuman',
 'tradisional',
 'menyegarkan',
 'menyehatkan',
 'tubuh',
 'provinsi',
 'jawa',
 'barat',
 'minuman',
 'khas',
 'wajib',
 'dicoba',
 'merangkum',
 'akun',
 'instagram',
 'dinas',
 'pendidikan',
 'jawa',
 'barat',
 'sabtu',
 'menggugah',
 'selera',
 'menyehatkan',
 'yuk',
 'baca',
 'informasi',
 'minuman',
 'terbuat',
 'gula',
 'aren',
 'santan',
 'kelapa',
 'disajikan',
 'hangat',
 'diminum',
 'cuaca',
 'dingin',
 'menambah',
 'kenikmatan',
 'khasiat',
 'tubuh',
 'bajigur',
 'dicampur',
 'jahe',
 'garam',
 'bajigur',
 'tubuh',
 'bahanbahan',
 'alami',
 'menyehatkan',
 'minuman',
 'memiliki',
 'aroma',
 'kuat',
 'memakai',
 'rempahrempah',
 'menyehatkan',
 'jahe',
 'gula',
 'merah',
 'bahan',
 'dasar',
 'bandrek',
 'menghangatkan',
 'tubuh',
 'bandrek',
 'mengobati',
 'penyakit

In [ ]:
wordfreq = {}
for word in clean:
    if word not in wordfreq:
        wordfreq[word] = 0 
    wordfreq[word] += 1

In [ ]:
wordfreq

In [ ]:
sorted(wordfreq.items(), key=lambda kv:(kv[1], kv[0]), reverse=True)

dump

In [ ]:
flat_list = [item for sublist in contents for item in sublist]

In [ ]:
stopwords.words('indonesian')

['ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 'berikut'